# **CA4. Multiprocessing**
**Sansores Cruz Angel David\
Data Engineering\
Universidad Politécnica de Yucatán\
Ucú, Yucatán, México\
2109139@upy.edu.mx** 

# Part 4. Matrix and Vector Computations

13. Read about Broadcasting with Arrays on the chapter Computation on Arrays:\
Broadcasting from Python Data Science Handbook (J. VandePlas, 2016):\
**Link:** https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computationon-arrays-broadcasting.html.

The integrated text highlights key features of NumPy for efficient computing, focusing on vectorized operations through universal functions (ufuncs) and the concept of **broadcasting**. It explains how ufuncs enable fast array computations, bypassing the characteristic slowness of loops in Python by utilizing both unary and binary operations, including arithmetic and trigonometric operations, absolute values, exponents, and logarithms. Moreover, specialized features of ufuncs such as output specification, aggregates, and outer products significantly enhance computation efficiency and capability in NumPy.

On the other hand, broadcasting stands out as a powerful feature that facilitates vectorized operations across arrays of different shapes without explicit data replication, simplifying the code and enhancing performance. The rules that NumPy follows to handle operations between mismatched shapes are described, such as extending dimensions of size one to match the corresponding dimensions of the other array, avoiding data duplication and promoting computational and memory efficiency essential for high-performance computing (HPC). These rules apply to any binary universal function in NumPy, highlighting its versatility for a wide range of mathematical operations.

Additionally, examples are provided to demonstrate how broadcasting supports common operations in data processing, scientific computing, and machine learning, such as centering an array by subtracting the mean or computing functions over a grid. In the context of HPC and matrix/vector calculations, broadcasting is crucial for the efficient execution of vectorized operations, key to achieving high performance in numerical computations, simplifying the coding and maintenance of matrix and vector operations, and offering flexibility in algorithm design and mathematical model implementation.

14. Read Rewriting the particle simulator in Numpy on Chapter 2: Fast Array Operations
with Numpy and Pandas (pp. 68) from the book G. Lenaro (2017). Python high
Performance. Second Edition. UK: Packt Publishing Ltd. Implement the
improvements on the particle simulator using NumPy. Show that both
implementations scale linearly with particle size, but the runtime in the pure Python
version grows much faster than the NumPy version. 

In [1]:
!pip install numpy

   ---------------------------------------- 97.0/97.0 kB 923.4 kB/s eta 0:00:00


In [ ]:
import numpy as np
from random import uniform

In [ ]:
class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel

class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)

        for _ in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.ang_vel * v_x
                d_y = timestep * p.ang_vel * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])

        for _ in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i

        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

def benchmark(npart=100, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)

    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

# Example of benchmarking
if __name__ == "__main__":
    from timeit import timeit

    print("Benchmarking with 100 particles:")
    print("Python version:", timeit("benchmark(100, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(100, 'numpy')", globals=globals(), number=1), "seconds")

15. Explain how to optain the optimal performance with numexpr. Read the section
Reaching optimal performance with numexpr, pp. 72 from the previous reference.
Implement it and measure the execution time.

NumPy enhances computational speed through array broadcasting and optimized memory utilization, particularly beneficial for handling large datasets. Benchmark comparisons highlight the efficiency gains when transitioning from pure Python loops to NumPy's vectorized operations, with the advantage growing alongside the dataset size.

Building on NumPy's foundation, Numexpr takes these improvements further by optimizing array expressions, minimizing memory consumption, and leveraging multiple cores for parallel processing. By circumventing the need for intermediate array storage and making effective use of CPU cache, Numexpr achieves notable speed enhancements in operations with large arrays.

These advancements are exemplified in scenarios such as calculating norms and particle displacements, where Numexpr's ability to compile and optimize complex expressions in real-time results in significant performance improvements over conventional NumPy operations.

In [ ]:
!pip install numexpr

In [ ]:
import numpy as np
from random import uniform
import numexpr as ne

In [ ]:
class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel
class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles
    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.ang_vel * v_x
                d_y = timestep * p.ang_vel * v_y
                p.x += d_x
                p.y += d_y
    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]
 
    def evolve_numexpr(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])
 
        for i in range(nsteps):
            norm_i = ne.evaluate("sqrt((r_i ** 2).sum(axis=1))")
            v_i = r_i[:, [1, 0]]
            v_i[:, 0] = ne.evaluate("-v_i[:, 0]")
            v_i = ne.evaluate("v_i / norm_i[:, None]")
            d_i = ne.evaluate("timestep * ang_vel_i[:, None] * v_i")
            r_i = ne.evaluate("r_i + d_i")
 
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]
 
def benchmark(npart=100000, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)
    elif method == 'numexpr':
        simulator.evolve_numpy(0.1)
 
# Example of benchmarking
if __name__ == "__main__":
    from timeit import timeit
    print("Benchmarking with 100000 particles:")
    print("Python version:", timeit("benchmark(100000, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(100000, 'numpy')", globals=globals(), number=1), "seconds")
    print("NumExpr version:", timeit("benchmark(100000, 'numexpr')", globals=globals(), number=1), "seconds")